In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
import string

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
train=train.drop('keyword',1)
train=train.drop('location',1)

test=test.drop('keyword',1)
test=test.drop('location',1)

In [5]:
y_train=train.iloc[:,-1]
x_train=train.iloc[:,:-1]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.2)

In [6]:
ps=PorterStemmer()
lemmatizer=WordNetLemmatizer()

In [29]:
def create_corpus(data):
    corpus=[]
    for i in range(data.shape[0]):
        tweet=data.iloc[i,-1]
        tweet=re.sub(r'http\S+', '', tweet)
        tweet=re.sub('[^a-zA-z]'," ",tweet)
        tweet=tweet.lower()
        tweet=word_tokenize(tweet)
#         tweet=[ps.stem(word) for word in tweet if word not in stopwords.words('english')]
        tweet=[lemmatizer.lemmatize(word) for word in tweet if word not in stopwords.words('english')]
        tweet=[word for word in tweet if word not in set(string.punctuation)]
        tweet=" ".join(tweet)
        corpus.append(tweet)
    return corpus

In [30]:
x_corpus_train=create_corpus(x_train)
x_corpus_test=create_corpus(test)

# x_corpus_train=create_corpus(X_train)
# x_corpus_test=create_corpus(X_test)

In [31]:
dic=defaultdict(int)
for text in x_corpus_train:
    text=text.split()
    for word in text:
        dic[word]=dic[word]+1

In [43]:
sorted(dic.items(), key=lambda x:x[1],reverse=True)[:8000]
# len(dic)

[('fire', 356),
 ('like', 350),
 ('amp', 344),
 ('u', 263),
 ('get', 255),
 ('new', 227),
 ('via', 220),
 ('news', 212),
 ('one', 207),
 ('people', 201),
 ('time', 181),
 ('year', 178),
 ('video', 174),
 ('disaster', 162),
 ('emergency', 159),
 ('body', 155),
 ('day', 151),
 ('home', 144),
 ('police', 143),
 ('building', 141),
 ('would', 137),
 ('family', 132),
 ('life', 131),
 ('go', 131),
 ('say', 131),
 ('still', 129),
 ('storm', 128),
 ('got', 125),
 ('crash', 124),
 ('back', 122),
 ('california', 121),
 ('burning', 121),
 ('bomb', 121),
 ('look', 120),
 ('know', 120),
 ('suicide', 120),
 ('man', 117),
 ('death', 117),
 ('world', 117),
 ('flood', 115),
 ('car', 115),
 ('train', 115),
 ('see', 112),
 ('attack', 110),
 ('first', 109),
 ('rt', 108),
 ('love', 107),
 ('two', 104),
 ('make', 104),
 ('going', 104),
 ('nuclear', 104),
 ('pm', 103),
 ('today', 102),
 ('let', 102),
 ('war', 101),
 ('killed', 99),
 ('dead', 98),
 ('accident', 98),
 ('youtube', 98),
 ('w', 95),
 ('want', 95),

In [50]:
# from sklearn.feature_extraction.text import CountVectorizer
# cv=CountVectorizer(max_features=5000)

from sklearn.feature_extraction.text import TfidfVectorizer
cv=TfidfVectorizer(max_features=8000)

In [51]:
x_train_vector=cv.fit_transform(x_corpus_train).toarray()
x_test_vector=cv.transform(x_corpus_test).toarray()

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(x_train_vector,y_train)
print(model.score(x_train_vector,y_train))
y_pred=model.predict(x_test_vector)
y_pred

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# rfc=RandomForestClassifier(random_state=4,n_estimators=500,warm_start=True,max_depth=6,min_samples_leaf=2,max_features='auto',min_samples_split=3)
# rfc.fit(x_train_vector,y_train)
# print(rfc.score(x_train_vector,y_train))
# y_pred=rfc.predict(x_test_vector)

In [ ]:
from xgboost import XGBClassifier
xgb=XGBClassifier()
xgb.fit(x_train_vector,y_train,early_stopping_rounds=5, 
             eval_set=[(x_train_vector,y_train)], 
             verbose=False)
print(xgb.score(x_train_vector,y_train))
y_pred=xgb.predict(x_test_vector)

In [52]:
from sklearn.linear_model import LogisticRegression
reg=LogisticRegression()
reg.fit(x_train_vector,y_train)
print(reg.score(x_train_vector,y_train))
y_pred=reg.predict(x_test_vector)
y_pred

# print(reg.score(x_test_vector,Y_test))

0.8761329305135952


array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [45]:
from sklearn.linear_model import PassiveAggressiveClassifier
passive=PassiveAggressiveClassifier()
passive.fit(x_train_vector,y_train)
print(passive.score(x_train_vector,y_train))
y_pred=passive.predict(x_test_vector)
y_pred

# print(passive.score(x_test_vector,Y_test))

0.9743859188230658


array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [46]:
# from sklearn.neighbors import KNeighborsClassifier
# classifier = KNeighborsClassifier(n_neighbors = 2)
# classifier.fit(x_train_vector,y_train)
# print(classifier.score(x_train_vector,y_train))
# y_pred=classifier.predict(x_test_vector)

In [53]:
submission=pd.read_csv('sample_submission.csv')
submission.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [54]:
submission['target']=y_pred

In [55]:
submission.to_csv('submission.csv',index=False)